In [ ]:
#引入相关模块
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
tf.random.set_seed(42)

In [ ]:
#读取数据
full_data = pd.read_excel('Volve_production_data.xlsx')
full_data

In [ ]:
column_names = full_data.columns
print(column_names)

In [ ]:
unique_well_bore = full_data['NPD_WELL_BORE_NAME'].unique()
print("所有井的名称:")
print(unique_well_bore)

unique_producer_well_bore = full_data[full_data['FLOW_KIND'] == 'production']['NPD_WELL_BORE_NAME'].unique()
print("\n产油井名称:")
print(unique_producer_well_bore)

unique_injection_well_bore = full_data[full_data['FLOW_KIND'] == 'injection']['NPD_WELL_BORE_NAME'].unique()
print("\n注水井名称:")
print(unique_injection_well_bore)


In [ ]:
full_data = full_data.sort_values(by='DATEPRD')

In [ ]:
# 使用全部产油井的数据
data = full_data[full_data['FLOW_KIND'] == 'production'][['DATEPRD', 'ON_STREAM_HRS', 'AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE', 'AVG_DP_TUBING', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'AVG_WHP_P', 'AVG_WHT_P', 'DP_CHOKE_SIZE', 'BORE_OIL_VOL']]

In [ ]:
print(f'We have {data.shape[0]} samples')

In [ ]:
null_values = data.isnull().sum()
print("Null values:")
print(null_values)

#### 缺失值处理方案一

删除缺失数据


In [ ]:
# data = data.dropna()
# null_values = data.isnull().sum()
# print("Null values:")
# print(null_values)

#### 缺失值处理方案二

- 少量缺失数据删除(<=15)
- 中等缺失(181-242)线性插值
- 大量缺失特征处理(油井平均压力填充)

In [ ]:
# 少量缺失数据删除
data = data.dropna(subset=['AVG_WHP_P', 'AVG_WHT_P', 'DP_CHOKE_SIZE']) 

# 中等缺失(181-242)线性插值 ['AVG_CHOKE_SIZE_P', 'AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE', 'AVG_DP_TUBING']
data[['AVG_CHOKE_SIZE_P', 'AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE', 'AVG_DP_TUBING']] = data[['AVG_CHOKE_SIZE_P', 'AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE', 'AVG_DP_TUBING']].interpolate()

# 大量缺失特征处理 油井平均压力填充
if 'AVG_ANNULUS_PRESS' in data.columns:
    data['AVG_ANNULUS_PRESS'].fillna(data['AVG_DOWNHOLE_PRESSURE'], inplace=True)

data.isnull().sum()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(15, 6))
sns.lineplot(data=data, x='DATEPRD', y='BORE_OIL_VOL',errorbar=None)
plt.xlabel('Date')
plt.ylabel('Oil Volume')
plt.title('Production Data - Date vs. Oil Volume')
plt.tight_layout()
plt.show()

In [ ]:
tf.random.set_seed(42)
from sklearn.preprocessing import MinMaxScaler

columns_to_scale = ['ON_STREAM_HRS', 'AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE', 'AVG_DP_TUBING', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'AVG_WHP_P', 'AVG_WHT_P', 'DP_CHOKE_SIZE', 'BORE_OIL_VOL']
data_to_scale = data[columns_to_scale].copy()
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_to_scale)
data_scaled_df = pd.DataFrame(data_scaled, columns=columns_to_scale)

# Add the 'DATEPRD' column as the first column. Later, we will use it for plotting purpose.
data_scaled_df.insert(0, 'DATEPRD', data['DATEPRD'])

In [ ]:
# 将数据转换成适合LSTM的numpy数据格式
features = data_scaled_df[['ON_STREAM_HRS', 'AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE', 'AVG_DP_TUBING', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'AVG_WHP_P', 'AVG_WHT_P', 'DP_CHOKE_SIZE', 'BORE_OIL_VOL']].to_numpy()
target = data_scaled_df[['BORE_OIL_VOL']].to_numpy()

In [ ]:
# Dataset split
from sklearn.model_selection import train_test_split

X_train, X_temp, Y_train, Y_temp = train_test_split(features, target, test_size=0.3, shuffle=False)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, shuffle=False)
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_val = np.array(X_val)
Y_val = np.array(Y_val)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

print("Number of samples in train set:", X_train.shape[0])
print("Number of samples in validation set:", X_val.shape[0])
print("Number of samples in test set:", X_test.shape[0])

In [ ]:
plt.figure(figsize=(15, 6))

sns.lineplot(x=data['DATEPRD'][:Y_train.shape[0]].ravel(), y=Y_train.ravel(), label='Train', errorbar=None)
sns.lineplot(x=data['DATEPRD'][Y_train.shape[0]:Y_train.shape[0] + Y_val.shape[0]].ravel(), y=Y_val.ravel(), label='Val', errorbar=None)
sns.lineplot(x=data['DATEPRD'][Y_train.shape[0] + Y_val.shape[0]:Y_train.shape[0] + Y_val.shape[0] + Y_test.shape[0]].ravel(), y=Y_test.ravel(), label='Test', errorbar=None)

plt.xlabel('Date')
plt.ylabel('Oil Volume')
plt.title('Production Data - Date vs. Oil Volume')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
def windowed_dataset(features, target, window_size, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((features, target))
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda x, y: tf.data.Dataset.zip((x.batch(window_size + 1), y.batch(window_size + 1))))
    dataset = dataset.map(lambda x, y: (x[:-1], y[1:]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
tf.random.set_seed(42)
window_size = 5
batch_size = 32
train_set = windowed_dataset(X_train, Y_train, window_size, batch_size)
val_set = windowed_dataset(X_val, Y_val, window_size, batch_size)
test_set = windowed_dataset(X_test, Y_test, window_size, batch_size)

In [ ]:
train_iterator = iter(train_set)
first_element = next(train_iterator)
features_shape = first_element[0].shape
target_shape = first_element[1].shape

print("Features shape:", features_shape)
print("Target shape:", target_shape)

In [ ]:
first_sample = (first_element[0][0], first_element[1][0])

for day in range(len(first_sample[0])):
    print(f"\033[1;34mDay {day + 1} Data:\033[0m")
    print(f"Input data for day {day + 1}: {first_sample[0][day]}")
    print(f"Target (oil production) for day {day + 2}: {first_sample[1][day]}")
    print()

In [ ]:
from typing import Tuple
from tensorflow.keras.layers import Bidirectional, LayerNormalization, Input
from tensorflow.keras.models import Model

class TemporalAttention(tf.keras.layers.Layer):
    """时间注意力机制层"""
    def __init__(self, units: int):
        super().__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, inputs):
        hidden_with_time_axis = tf.expand_dims(inputs, 1)
        score = self.V(tf.nn.tanh(
            self.W1(inputs) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * inputs
        return tf.reduce_sum(context_vector, axis=1)

def create_bilstm_attention_model(input_shape: Tuple[int, int]) -> tf.keras.Model:
    """
    创建动态输入形状的BiLSTM+Attention模型
    
    参数:
        input_shape: (window_size, num_features) 输入数据的形状
        
    返回:
        编译好的Keras模型实例
    """
    # 输入层
    inputs = tf.keras.Input(shape=input_shape)
    
    # 双向LSTM层
    bilstm = Bidirectional(
        tf.keras.layers.LSTM(128, return_sequences=True),
        merge_mode='concat')(inputs)
    
    # 层标准化
    norm = LayerNormalization()(bilstm)
    
    # Attention机制
    attention = TemporalAttention(64)(norm)
    
    # 输出层
    outputs = tf.keras.layers.Dense(1)(attention)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# 使用示例:
model = create_bilstm_attention_model(input_shape=(window_size, features.shape[1]))  # 5天窗口,10个特征
model.summary()

In [ ]:
# 训练配置优化版
tf.random.set_seed(42)

# 动态学习率调度
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,  # 稍大的初始学习率
    decay_steps=500,
    decay_rate=0.9)

# 早停策略调整
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=100,
    restore_best_weights=True,
    min_delta=0.001)  # 最小改进阈值

# 增加指标监控
metrics = [
    tf.keras.metrics.MeanAbsoluteError(name='mae'),
    tf.keras.metrics.RootMeanSquaredError(name='rmse')
]

model.compile(
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=metrics)

# 增加模型检查点
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    save_best_only=True,
    monitor='val_loss')

history = model.fit(
    train_set,
    epochs=500,  # 减少总轮次
    validation_data=val_set,
    shuffle=False,  # 保持时序数据特性
    callbacks=[early_stopping, checkpoint],
    verbose=1)


In [ ]:
train_loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(test_set)

In [ ]:
# 获取最后一步的预测值（假设预测的是每个窗口的最后一天）
predictions_flat = predictions[:, -1, :]  # 形状从(1121,5,1)变为(1121,1)

# 确保X_test切片正确
test_features = X_test[window_size:, 1:]  # 跳过日期列，从window_size开始

# 创建DataFrame时确保维度匹配
df_pred = pd.concat([
    pd.DataFrame(test_features), 
    pd.DataFrame(predictions_flat, columns=['PREDICTED'])
], axis=1)

# 逆缩放时确保列顺序与scaler训练时一致
rev_trans = scaler.inverse_transform(df_pred)

# 获取对应时间段的原始数据
df_final = data.iloc[-len(predictions_flat):].copy()
df_final['PREDICTED_OIL'] = rev_trans[:, -1]  # 取最后一列(预测值列)


In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_final, x='DATEPRD', y='BORE_OIL_VOL', label='Actual', errorbar=None)
sns.lineplot(data=df_final, x='DATEPRD', y='PREDICTED_OIL', label='Predicted', errorbar=None)
plt.xlabel('Date')
plt.ylabel('Oil Production')
plt.title('Actual vs. Predicted')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
compare_data = df_final[['DATEPRD', 'BORE_OIL_VOL', 'PREDICTED_OIL']]
compare_data

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# 假设 compare_data 是包含真实值和预测值的 DataFrame
# compare_data = df_final[['DATEPRD', 'BORE_OIL_VOL', 'PREDICTED_OIL']]

# 提取真实值和预测值
y_true = compare_data['BORE_OIL_VOL']
y_pred = compare_data['PREDICTED_OIL']

# 计算 MSE
mse = mean_squared_error(y_true, y_pred)
print(f"均方误差 (MSE): {mse}")

# 计算 MAE
mae = mean_absolute_error(y_true, y_pred)
print(f"平均绝对误差 (MAE): {mae}")

# 计算 RMSE
rmse = np.sqrt(mse)
print(f"均方根误差 (RMSE): {rmse}")

# 计算 R²
r2 = r2_score(y_true, y_pred)
print(f"决定系数 (R²): {r2}")